In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import pickle
import matplotlib.pyplot as plt

# Datos Tasa de Incidencia Covid

In [22]:
basepath = "/media/jonathan/JHON/Tesis/DatosTesis/Data_Covid19/covid19FEB_JUL.csv"
data = pd.read_csv(basepath, sep=';',dtype={"municipio_distrito": str,
                                        "tasa_incidencia_acumulada_ultimos_14dias": str,
                                        "tasa_incidencia_acumulada_total": str},parse_dates=['fecha_informe'],usecols=[0,1,3], encoding='latin-1')

data['TasaIncidenciaAcumulada_Ultimos14dias'] = [int(value.split(",",1)[0]) for value in data['tasa_incidencia_acumulada_ultimos_14dias']]
data['fecha_informe'] = data['fecha_informe'].apply(lambda x:x.replace(hour=0,minute=0))
#data['fecha_informe'] = data['fecha_informe'].dt.strftime("%Y-%m-%d %H:%M:%S")
data.set_index('fecha_informe', inplace = True)



basepath = "/media/jonathan/JHON/Tesis/DatosTesis/Data_Covid19/covid19JUL_2022.csv"
data2 = pd.read_csv(basepath, sep=';',dtype={"municipio_distrito": str,
                                        "tasa_incidencia_acumulada_ultimos_14dias": str,
                                        "tasa_incidencia_acumulada_total": str},parse_dates=['fecha_informe'],usecols=[0,1,5], encoding='latin-1')

data2['TasaIncidenciaAcumulada_Ultimos14dias'] = [int(value.split(",",1)[0]) for value in data2['tasa_incidencia_acumulada_ultimos_14dias']]
data2['fecha_informe'] = data2['fecha_informe'].apply(lambda x:x.replace(hour=0,minute=0))
data2.set_index('fecha_informe', inplace = True)

data2.head(2)

,municipio_distrito,tasa_incidencia_acumulada_ultimos_14dias,TasaIncidenciaAcumulada_Ultimos14dias
fecha_informe,,,
2022-03-29,Madrid-Retiro,"301,6",301
2022-03-29,Madrid-Salamanca,"266,5",266


# Tablas Tasa de Incidencia Covid

In [19]:
table = pd.pivot_table(data, values=["TasaIncidenciaAcumulada_Ultimos14dias"], index=["fecha_informe"], columns=["municipio_distrito"], aggfunc={"TasaIncidenciaAcumulada_Ultimos14dias":"first"})
table.head(5)

table2 = pd.pivot_table(data2, values=["TasaIncidenciaAcumulada_Ultimos14dias"], index=["fecha_informe"], columns=["municipio_distrito"], aggfunc={"TasaIncidenciaAcumulada_Ultimos14dias":"first"})
table2.head(5)

table = table.astype(np.float64)
table2 = table2.astype(np.float64)

#Quitamos los niveles sobrantes del nivel superior y reiniciamos index - Solo se hace una vez de lo contrario genera problema
table.columns =[str(s2) for (s1,s2) in table.columns.tolist()]
table_Final = table.reset_index()
table_Final.set_index('fecha_informe', inplace = True)

table2.columns =[str(s2) for (s1,s2) in table2.columns.tolist()]
table2_Final = table2.reset_index()
table2_Final.set_index('fecha_informe', inplace = True)

#table.dtypes
#table.iloc[120:127,97:115]
table_Final.head(2)

,Ajalvir,Alameda del Valle,Alcalá de Henares,Alcobendas,Alcorcón,Aldea del Fresno,Algete,Alpedrete,Ambite,Anchuelo,...,Villamanta,Villamantilla,Villanueva de Perales,Villanueva de la Cañada,Villanueva del Pardillo,Villar del Olmo,Villarejo de Salvanés,Villaviciosa de Odón,Villavieja del Lozoya,Zarzalejo
fecha_informe,,,,,,,,,,,,,,,,,,,,,
2020-02-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Procesamiento datos desplazamiento por trayecto

In [5]:
def df_origen(origen,destino):     
    
    ######################PICKLE###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names2.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)

    df = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Referencia2.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)    
    
    ######################PICKLE 1###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento1.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df1 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento1.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    ######################PICKLE 2###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento2.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df2 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento2.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    ######################PICKLE 3###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento3.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df3 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento3.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    ######################PICKLE 4###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento4.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df4 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento4.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    ######################PICKLE 5###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento5.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df5 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento5.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    ######################PICKLE 6###############################
    with open('/home/jonathan/tesis/3er_avance/columns_names_confinamiento6.pickle', "rb") as file:   
        columns_names = pickle.load(file)

    names = list(columns_names)
    lista=["ds"]

    for trayectos in names:    
        x=trayectos.startswith(origen)    
        if x==True:                         
            lista.append(trayectos)
    
    df6 = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento6.csv", sep=',',index_col=0, parse_dates=True, usecols=lista)
    
    #Filtramos el destino
    columns_names_df = df.columns   
    names_1 = list(columns_names_df)   
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)            
    df_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Referencia2.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)    
    
    #Cambiamos el nombre de las columnas
    lines_names = {"19024":"Fuera_Madrid","2800601":"Fuera_Madrid","2806504":"Fuera_Madrid","2807901":"Centro","2807902":"Arganzuela","2807903":"Retiro","2807904":"Salamanca","2807905":"Chamartín","2807906":"Tetuán","2807907":"Chamberí","2807908":"El_Pardo","2807909":"Moncloa-Aravaca","2807910":"Latina","2807911":"Carabanchel","2807912":"Usera","2807913":"Puente_de_Vallecas","2807914":"Moratalaz","2807915":"Ciudad_Lineal","2807916":"Hortaleza","2807917":"Villaverde","2807918":"Villa_de_Vallecas","2807919":"Vicálvaro","2807920":"San Blas-Canillejas","2812301":"Fuera_Madrid","2807921":"Barajas"}            
    df_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df_Final['Fecha'] = df_Final.index.date
    df_Final['Año'] = df_Final.index.year                                
    df_Final['Mes'] = df_Final.index.month                               
    df_Final['ID_Dia'] = df_Final.index.dayofweek
    df_Final['Hora'] = df_Final.index.hour
    df_Final['WeekNo'] = df_Final.index.isocalendar().week
    df_Final[lines_names[origen]] = table_Final["Madrid"+"-"+lines_names[origen]].loc['2020-02-14':'2020-03-16']
          
    
    #Filtramos el destino
    columns_names_df = df1.columns   
    names_1 = list(columns_names_df)    
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)
    df1_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento1.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)
    
    #Cambiamos el nombre de las columnas    
    df1_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df1_Final['Fecha'] = df1_Final.index.date
    df1_Final['Año'] = df1_Final.index.year                                
    df1_Final['Mes'] = df1_Final.index.month                               
    df1_Final['ID_Dia'] = df1_Final.index.dayofweek
    df1_Final['Hora'] = df1_Final.index.hour
    df1_Final['WeekNo'] = df1_Final.index.isocalendar().week
    df1_Final[lines_names[origen]] = table_Final["Madrid"+"-"+lines_names[origen]].loc['2020-03-17':'2020-05-31']    
    

    #Filtramos el destino
    columns_names_df = df2.columns   
    names_1 = list(columns_names_df)    
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)
    df2_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento2.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)
    
    #Cambiamos el nombre de las columnas    
    df2_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df2_Final['Fecha'] = df2_Final.index.date
    df2_Final['Año'] = df2_Final.index.year                                
    df2_Final['Mes'] = df2_Final.index.month                               
    df2_Final['ID_Dia'] = df2_Final.index.dayofweek
    df2_Final['Hora'] = df2_Final.index.hour
    df2_Final['WeekNo'] = df2_Final.index.isocalendar().week   
    df2_Final[lines_names[origen]] = table2_Final["Madrid"+"-"+lines_names[origen]].loc['2020-06-01':'2020-07-31']
    
    
    #Filtramos el destino
    columns_names_df = df3.columns   
    names_1 = list(columns_names_df)    
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)
    df3_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento3.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)
    
    #Cambiamos el nombre de las columnas    
    df3_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df3_Final['Fecha'] = df3_Final.index.date
    df3_Final['Año'] = df3_Final.index.year                                
    df3_Final['Mes'] = df3_Final.index.month                               
    df3_Final['ID_Dia'] = df3_Final.index.dayofweek
    df3_Final['Hora'] = df3_Final.index.hour
    df3_Final['WeekNo'] = df3_Final.index.isocalendar().week
    df3_Final[lines_names[origen]] = table2_Final["Madrid"+"-"+lines_names[origen]].loc['2020-08-01':'2020-10-31']
    
    
    #Filtramos el destino
    columns_names_df = df4.columns   
    names_1 = list(columns_names_df)    
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)
    df4_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento4.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)
    
    #Cambiamos el nombre de las columnas    
    df4_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df4_Final['Fecha'] = df4_Final.index.date
    df4_Final['Año'] = df4_Final.index.year                                
    df4_Final['Mes'] = df4_Final.index.month                               
    df4_Final['ID_Dia'] = df4_Final.index.dayofweek
    df4_Final['Hora'] = df4_Final.index.hour
    df4_Final['WeekNo'] = df4_Final.index.isocalendar().week   
    df4_Final[lines_names[origen]] = table2_Final["Madrid"+"-"+lines_names[origen]].loc['2020-11-01':'2020-12-31']
    
    
    #Filtramos el destino
    columns_names_df = df5.columns   
    names_1 = list(columns_names_df)   
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)            
    df5_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento5.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)    
    
    #Cambiamos el nombre de las columnas    
    df5_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df5_Final['Fecha'] = df5_Final.index.date
    df5_Final['Año'] = df5_Final.index.year                                
    df5_Final['Mes'] = df5_Final.index.month                               
    df5_Final['ID_Dia'] = df5_Final.index.dayofweek
    df5_Final['Hora'] = df5_Final.index.hour
    df5_Final['WeekNo'] = df5_Final.index.isocalendar().week    
    df5_Final[lines_names[origen]] = table2_Final["Madrid"+"-"+lines_names[origen]].loc['2021-01-01':'2021-01-31']
    

    #Filtramos el destino
    columns_names_df = df6.columns   
    names_1 = list(columns_names_df)   
    listaDst=["ds"]
    for trayectos in names_1:            
        x=trayectos.split("-",1)[1]        
        if x==destino:                         
            listaDst.append(trayectos)            
    df6_Final = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento6.csv", sep=',',index_col=0, parse_dates=True, usecols=listaDst)        
    
    #Cambiamos el nombre de las columnas    
    df6_Final.columns = [lines_names[origen]+"-"+lines_names[destino]]    
    
    #Añadimos columnas al Dataframe
    df6_Final['Fecha'] = df6_Final.index.date
    df6_Final['Año'] = df6_Final.index.year                                
    df6_Final['Mes'] = df6_Final.index.month                               
    df6_Final['ID_Dia'] = df6_Final.index.dayofweek
    df6_Final['Hora'] = df6_Final.index.hour
    df6_Final['WeekNo'] = df6_Final.index.isocalendar().week  
    df6_Final[lines_names[origen]] = table2_Final["Madrid"+"-"+lines_names[origen]].loc['2021-02-01':'2021-05-09']
    
    
    return df_Final, df1_Final, df2_Final, df3_Final, df4_Final, df5_Final, df6_Final, listaDst

origen = "2807905"
destino = "2807901"
df_Final, df1_Final, df2_Final, df3_Final, df4_Final, df5_Final, df6_Final, listaDst = df_origen(origen,destino)
df_Final.tail(2)

,Chamartín-Centro,Fecha,Año,Mes,ID_Dia,Hora,WeekNo,Chamartín
ds,,,,,,,,
2020-03-16 22:00:00,55.801,2020-03-16,2020,3,0,22,12,NaN
2020-03-16 23:00:00,21.306,2020-03-16,2020,3,0,23,12,NaN


In [18]:
#df6_Final[2040:2100]

In [16]:
#table2_Final.iloc[40:100,88:200]

In [17]:
#df_Final.iloc[288:348]